In [47]:
import struct

import pandas as pd


In [48]:
# Compresses it
for i in range(0, 24):
    df = pd.read_csv(f"data/time_{i}_to_{i+1}.txt") # From file
    df = df[['x', 'y', 'z']].copy()
    format_str = 'ddd'
    file_path = f"data/time_{i}_to_{i+1}_compressed.bin" # To file
    with open(file_path, "wb") as fp:
        for i, row in enumerate(df.itertuples(index=False)):
            packed_data = struct.pack(format_str, row.x, row.y, row.z)
            fp.write(packed_data)
            

In [49]:
# Read orders back from the file
import struct

def deserialize_data(packed_data):
    """
    Deserialize bytes to an order record.
    """
    format_str = 'ddd'  # int, 50-char string, 100-char string, float
    unpacked_data = struct.unpack(format_str, packed_data)
    
    record = {
        'x': unpacked_data[0],
        'y': unpacked_data[1],
        'z': unpacked_data[2],
    }
    return record

format_str = 'ddd'
data_size = struct.calcsize('ddd')
filename = "data/time_10_to_11_compressed.bin"
all_data = []
with open(filename, 'rb') as file:
    while True:
        # Read a chunk of data of size order_size
        binary_data = file.read(data_size)
        if not binary_data:  # Break if no more data is available
            break
        unpacked_order = deserialize_data(binary_data)
        all_data.append(unpacked_order)
df = pd.DataFrame(all_data)
df


,x,y,z
0,30.00000,30.000000,66.743746
1,30.00000,30.051014,17.208061
2,30.00000,30.102028,83.483826
3,30.00000,30.153042,18.228844
4,30.00000,30.204056,45.067934
...,...,...,...
962356,79.99376,79.789704,85.819339
962357,79.99376,79.840718,49.208536
962358,79.99376,79.891732,3.111700
962359,79.99376,79.942746,65.897616


### Validate that bin and txt are the same

In [50]:
df2 = pd.read_csv("data/time_10_to_11.txt")
df2 = df2[['x', 'y', 'z']].copy()
df2


,x,y,z
0,30.00000,30.000000,66.743746
1,30.00000,30.051014,17.208061
2,30.00000,30.102028,83.483826
3,30.00000,30.153042,18.228844
4,30.00000,30.204056,45.067934
...,...,...,...
962356,79.99376,79.789704,85.819339
962357,79.99376,79.840718,49.208536
962358,79.99376,79.891732,3.111700
962359,79.99376,79.942746,65.897616


In [51]:
def are_dataframes_identical(df1, df2):
    # Check shape
    if df1.shape != df2.shape:
        return False, "Shape mismatch"
    
    # Check column names and order
    if not df1.columns.equals(df2.columns):
        return False, "Column names or order mismatch"
    
    # Check data types
    if not df1.dtypes.equals(df2.dtypes):
        return False, "Data types mismatch"
    
    # Check index
    if not df1.index.equals(df2.index):
        return False, "Index mismatch"
    
    # Compare data
    if not df1.equals(df2):
        return False, "Data mismatch"
    
    return True, "DataFrames are identical"

are_dataframes_identical(df.sort_values(by=['x', 'y', 'z']), df2.sort_values(by=['x', 'y', 'z']))

(True, 'DataFrames are identical')

### Validate again

In [53]:
df1_review, df2_review = df.sort_values(by=['x', 'y', 'z']), df2.sort_values(by=['x', 'y', 'z'])
import numpy as np
import tqdm

for row1, row2 in tqdm.tqdm(zip(df1_review.itertuples(), df2_review.itertuples())):
    if np.round(row1.x, 10) != np.round(row2.x, 10) or np.round(row1.y, 10) != np.round(row2.y, 10) or np.round(row1.z, 10) != np.round(row2.z, 10):
        print("Error", row1.x, row2.x, row1.y, row2.y, row1.z, row2.z)

962361it [00:24, 39443.48it/s]
